# Guia para a aula experimental - Precisão

Nesse guia, trataremos passo-a-passo a conexão do dispositivo PK2 com a plataforma Konker, aquisição de dados de temperatura e análise da precisão da medida.
Para seguir esse roteiro, você precisará de uma conta na Konker, que pode ser criada em https://demo.konkerlabs.net/ e, para trabalhar com o kit disponibilizado para vocês, será necessário baixar a Arduino IDE e adicionar o suporte à placa de desenvolvimento **NodeMCU**, além de instalar as bibliotecas **PubSubClient** e **ArduinoJSON**. Um guia de instalação da placa pode ser visto na referência https://www.filipeflop.com/blog/programar-nodemcu-com-ide-arduino/, enquanto a instalação das bibliotecas pode ser feita buscando pelo nome das mesmas na caixa aberta ao selecionar o menu **Sketch** -> **Incluir Biblioteca** -> **Gerenciar Bibliotecas**.

**Por favor, siga o procedimento desse guia (python notebook) e lembre-se sempre de salvar suas alterações (o atalho Ctrl+s funciona). Ao termina-la, por favor publique seus resultados no Classrom, como indicado na última célula.**

Vamos iniciar nosso notebook chamando todas as bibliotecas que usaremos. Nessa etapa, basta rodar a célula abaixo com o comando **Shift + Enter**

In [ ]:
#Caso você rode o notebook no Google Colab, é necessário primeiro instalar a biblioteca arrow:
!pip install arrow

import matplotlib.pyplot as plt
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import pprint
import numpy as np
import arrow
import requests                                                                                                                                                                                 
import json                                                                                                                                                                                     
from threading import Timer

O próximo passo é definir os endereços que serão usados para consultar e enviar os dados. Nessa etapa, ainda não é necessário fazer modificações.

In [ ]:
#Url de publicacao dos dados
pub_url = 'https://data.demo.konkerlabs.net/pub/'
#Url da API
base_api = 'https://api.demo.konkerlabs.net'
#Application padrão
application = 'default'

Agora vamos colocar o seu usuário e senha da plataforma em duas variáveis:

In [ ]:
username = ''
password = ''

O próximo passo é criar (caso ele ainda não exista) o dispositivo **termometro** dentro de sua conta na Konker.
Quando você criar o dispositivo, clique em **Connect** -> **Generate Password** e coloque as credenciais do dispositivo nas variáveis abaixo:

In [ ]:
termometro_name = "termometro"
termometro_username = ""
termometro_password = ""

## Usando o Kit de Hardware PK2
Esse é o momento de você baixar o código do dispositivo disponível em nosso GitHub: https://github.com/KonkerLabs/arduino_examples/tree/master/Placa%20PK02/Termometro. Depois de baixar o código, abra no Arduino IDE e mude os parâmetros de rede Wifi, canal de publicação (minha sugestão é usar *temperatura*) e credenciais do dispositivo. Dica: você acabou de escrever essas credenciais nas variáveis acima como "cola" para esse momento, pois elas não serão usadas no notebook :-).

Agora é hora de configurar o seu termômetro!
Basta seguir a ilustração abaixo, colocando o jumper na posição T:

![term](https://raw.githubusercontent.com/KonkerLabs/arduino_examples/master/Placa%20PK02/Termometro/PK2-Exp2.png "Termômetro")

Com o dispositivo configurado, o próximo passo é compilar e gravar o Firmware. Lembre-se de mudar a board na Arduino IDE para **NodeMCU v1.0**.

Caso tudo tenha dado certo até o momento, você começará a observar os dados sendo enviados para a plataforma. Entre na Guia de Devices e procure **Messages** do seu dispositivo **termometro**. 
Você deve ver os dados de temperatura chegando, então não mexa mais na placa, pois vamos fazer o experimento do dia. :-)

## Usando a API
Para iniciar esse trabalho, vamos primeiro conectar na API da Konker. A API usa OAuth2, então, primeiro vamos obter o token de acesso. Caso você tenha inserido o seu usuário e senha da plataforma no início desse guia, essa célula deve rodar sem necessidade de modificações.

In [ ]:
client = BackendApplicationClient(client_id=username)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='{}/v1/oauth/token'.format(base_api),
                                       client_id=username,
                                       client_secret=password)

Ótimo! Agora nós podemos começar a usar a API. Caso você queira mais detalhes de sua utilização, pode encontrar a documentação em: https://api.demo.konkerlabs.net.

Vamos começar listando os dispositivos registrados no seu usuário.

In [ ]:
devices = oauth.get("https://api.demo.konkerlabs.net/v1/{}/devices/".format(application)).json()['result']
for dev in devices:
    print(dev)

Vamos procurar pelo dispositivo **termometro** na sua lista de dispositivos:

In [ ]:
guid_term=""
for dev in devices:
    if dev['name'] == termometro_name:
        guid_term = dev['guid']

print("O GUID do dispositivo Termômetro é: "+ guid_term)

Caso você consiga ver o GUID do dispositivo, significa que está tudo funcionando bem. Caso o GUID não apareça, revise o nome do dispositivo no notebook e o nome escolhido na plataforma para garantir que eles possuam a mesma grafia.

**Agora é a hora de você tomar um café :-)**

Cheque primeiramente se os dados estão chegando na plataforma. Caso afirmativo, deixe o dispositivo rodando sem intervenção por uns 60 minutos. Esse tempo é necessário para estabilizar a temperatura do termistor, que, por estar na placa, vai se aquecer com o uso do microcontrolador. 

Após aguardar esse tempo, vamos usar a API para pegar os dados enviados pelo dispositivo **termometro** na última hora. Caso você tenha escolhido outro canal para envio dos dados, por favor, modifique a variável **canal** na próxima célula.

In [ ]:
canal = 'temperatura'
dt_start = arrow.utcnow().to('America/Sao_Paulo').shift(hours=-1)
stats = oauth.get("https://api.demo.konkerlabs.net/v1/{}/incomingEvents?q=device:{} channel:{} timestamp:>{}&sort=oldest&limit=10000".format(application,guid_term,canal,dt_start.isoformat())).json()['result']
print(stats)

Caso tudo tenha funcionado como esperado, você deve estar vendo seus dados de temperatura logo acima. Para facilitar a visualização e análise dos dados, podemos transformar em um formato tabular com o Pandas.

In [ ]:
from pandas.io.json import json_normalize
stats_df = json_normalize(stats).set_index('timestamp')
stats_df = stats_df
stats_df

Ótimo! Agora os dados estão em um formato mais fácil de ler. Mas podemos também fazer um gráfico bem simples!

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(15,8))
plt.rcParams.update({'font.size': 18})
stats_df['payload.value'].plot()
plt.xticks(rotation=45)
plt.title('Gráfico da temperatura obtida com o termistor da placa PK02', size=24,y=1.02)
plt.ylabel('Temperatura [C]', size=24)
plt.xlabel('Data', size=24)
plt.grid(True)
plt.show()

Neste momento, começa a parte final desse trabalho. Vamos gerar um histograma com os valores obtidos:

In [ ]:
minimo = np.min(stats_df['payload.value'])
maximo = np.max(stats_df['payload.value'])
bins=np.arange(minimo-0.5,maximo+1,0.5)
hist, bin_edges = np.histogram(stats_df['payload.value'],bins=bins)
x = (bin_edges[:-1]+bin_edges[1:])/2

Vamos visualizar o histograma calculado com os dados:

In [ ]:
plt.figure(figsize=(15,8))
plt.step(x,hist,where='mid')
plt.title('Histograma da temperatura obtida com o termistor da placa PK02', size=24,y=1.02)
plt.ylabel('Número de contagens', size=24)
plt.xlabel('Temperatura [C]', size=24)
plt.grid(True)
plt.show()

Vamos tentar agora fazer um ajuste gaussiano nos dados:

In [ ]:
from scipy.optimize import curve_fit

#Definindo a função que usaremos para o ajuste
def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

In [ ]:
popt,pcov = curve_fit(gaus,x,hist,p0=[1,(minimo+maximo)/2,(maximo-minimo)])

In [ ]:
plt.figure(figsize=(15,8))
plt.step(x,hist,where='mid', label='Dados')
plt.plot(np.arange(minimo-0.5,maximo+0.5,0.01), gaus(np.arange(minimo-0.5,maximo+0.5,0.01), *popt), 'r-', label='Ajuste')
plt.title('Histograma da temperatura obtida com o termistor da placa PK02', size=24,y=1.02)
plt.ylabel('Número de contagens', size=24)
plt.xlabel('Temperatura [C]', size=24)
plt.grid(True)
plt.legend()
plt.show()

A partir do ajuste gaussiano, podemos obter o desvio padrão dos dados. Observe que esse número é influenciado não apenas pela precisão intrínseca do sensor, mas também pela escolha do "passo" (**bin**) escolhido para o histograma.

In [ ]:
print('Valor do desvio padrão obtido via ajuste Gaussiano: ' + str((popt[2]**2)**0.5))

Usando esse valor, podemos dizer que, dentro de um nível de confiança de **3 sigmas, ou seja, aproximadamente 99%**, esse sensor tem a seguinte precisão:

In [ ]:
precision = round((3*(popt[2]**2)**0.5),1)
print('Precisão (nível de confiança de 3 sigma): ' + str(precision) + ' C')

Observe também que é possível obter o desvio padrão sem o uso de um histograma, tornando o resultado independente da escolha de "passo". Usando o resultado direto do desvio padrão, deve-se obter uma precisão um pouco melhor (embora não muito diferente) da observada na célula acima:

In [ ]:
precision = round(3*(np.std(stats_df['payload.value'])),1)
print('Precisão (nível de confiança de 3 sigma): ' + str(precision) + ' C')

**Assim, finalizamos essa atividade! Se você conseguiu concluí-la, coloque seus resultados na atividade do Classroom "Atividades Experimentais", para trocarmos informações a respeito dos nossos sensores espalhados pelo Brasil!**

**Caso não tenha conseguido colcluí-la, procure ajuda de seus colegas, monitor ou professor no Classrom. :-)**